In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My \Drive/NLP

In [ ]:
!pip install textattack==0.3.7

In [ ]:
!pip install lime

In [ ]:
!pip install shap

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from lime.lime_text import LimeTextExplainer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import random
import transformers
import textattack
from textattack.datasets import Dataset
from datasets import load_dataset
from transformers import GPT2Tokenizer
from textattack.models.wrappers import ModelWrapper
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack import Attack
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance
from transformers import BertTokenizer
import shap

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# Load the dataset
df = pd.read_csv('./Data/KaggleData.csv')

# Convert to lowercase, remove punctuation, extra spaces, URLs, mentions, and hashtags
df['tweet'] = df['tweet'].str.lower().replace(r'[^\w\s]', '', regex=True).replace(' {2,}', ' ', regex=True).replace('"', '')
df['tweet'] = df['tweet'].replace(r'http\S+|www.\S+|@\w+|#\w+', '', regex=True)

# Tokenization
nltk.download('punkt')
df['tweet'] = df['tweet'].apply(nltk.word_tokenize)

# Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['tweet'] = df['tweet'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Removing stop-words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x if word not in stop_words]))

# Create a custom dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Encode the labels
# 0 - hate speech, 1 - offensive language, 2 - neither as positive or negative
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['class'])

# Splitting the Data using Stratified split
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], y, test_size=0.3, stratify=y, random_state=42)

# Tokenize and pad the input sequences
def tokenize_and_pad(texts, maxlen=100):
    tokenized_texts = [nltk.word_tokenize(text) for text in texts]
    return pad_sequence([torch.tensor([word2index[word] for word in text if word in word2index][:maxlen]) for text in tokenized_texts], batch_first=True, padding_value=len(word2index))

word2index = {word: i for i, word in enumerate(set(df['tweet'].str.cat(sep=' ').split()), 1)}
X_train = tokenize_and_pad(X_train)
X_test = tokenize_and_pad(X_test)

# Create PyTorch Datasets and DataLoaders
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Create a PyTorch LSTM model
class LSTMBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        packed_output, (hidden, cell) = self.lstm(x)
        x = self.fc(hidden[-1])
        return x

# Initialize the model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMBaseline(len(word2index) + 1, 50, 100, len(set(y))).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

    # Save Model
    torch.save(model, './Weights/KaggleLSTM_SHAP.pth')

model = torch.load('./Weights/KaggleLSTM_SHAP.pth')

# Test the model and collect predictions and true labels
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy, precision, recall, F1-score, and confusion matrix
accuracy = np.mean(np.array(predictions) == np.array(true_labels))
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
conf_mat = confusion_matrix(true_labels, predictions)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)
print("Confusion Matrix:\n", conf_mat)

In [ ]:
# Wrapper for the TextCNN model
class LSTMBaselineWrapper:
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        preds = []
        for text in text_input_list:
            input_tensor = tokenize_and_pad([text]).long()
            output = self.model(input_tensor.to(device))
            pred = torch.softmax(output, dim=1).squeeze().tolist()
            preds.append(pred)
        return np.array(preds)

def tokenize_and_pad(text_list):
    max_length = 50
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = [tokenizer.tokenize(text)[:max_length] for text in text_list]
    token_indices = np.zeros((len(tokens), max_length), dtype=int)
    for i, tweet in enumerate(tokens):
        for j, word in enumerate(tweet):
            if word in word2index:
                token_indices[i, j] = word2index[word]
    return torch.tensor(token_indices)

wrapped_model = LSTMBaselineWrapper(model)
class_names = ['hate_speech', 'offensive_language', 'neither']

# Explainability with SHAP
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def shap_analysis(text, wrapped_model, class_names):
    # Define a masker
    masker = shap.maskers.Text(tokenizer)

    # Initialize explainer
    explainer = shap.Explainer(wrapped_model, masker=masker)

    # Explain prediction
    shap_values = explainer([text])

    return shap_values

df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))
text_to_explain = random.choice(df['tweet'])
print("Text to explain:", text_to_explain)
shap_results = shap_analysis(text_to_explain, wrapped_model, class_names)

# Get predicted label for the text
predicted_label = np.argmax(wrapped_model([text_to_explain])[0])

shap.plots.text(shap_results[0])

def calculate_doe(shap_values, predicted_label):
    feature_scores = [abs(value) for value in shap_values.values[0][:, predicted_label]]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

doe = calculate_doe(shap_results, predicted_label)
print("Degree of Explainability (DoE):", doe)

# Define number of samples for analysis
num_sam = 20

# Load dataset
def load_custom_dataset(path):
    df = pd.read_csv(path)
    df = df.dropna(subset=['tweet', 'class'])
    return df

# SHAP Analysis
def shap_analysis(text, wrapped_model, class_names):
    # Define a masker
    masker = shap.maskers.Text(tokenizer)

    # Initialize explainer
    explainer = shap.Explainer(wrapped_model, masker=masker)

    # Explain prediction
    shap_values = explainer([text])

    return shap_values

# Calculate Degree of Explainability (DoE)
def calculate_doe(shap_values, predicted_label):
    feature_scores = [abs(value) for value in shap_values.values[0][:, predicted_label]]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

# Calculate average DoE for multiple samples
def calculate_average_doe(df, wrapped_model, class_names, samples=num_sam):
    doe_values = []
    sample_texts = random.sample(list(df['tweet']), samples)
    for text in sample_texts:
        shap_results = shap_analysis(text, wrapped_model, class_names)
        predicted_label = np.argmax(wrapped_model([text])[0])
        doe = calculate_doe(shap_results, predicted_label)
        doe_values.append(doe)
    average_doe = np.mean(doe_values)
    return average_doe

# Path to the dataset
test_data_path = "./Data/KaggleData.csv"
df = load_custom_dataset(test_data_path)

# Assuming wrapped_model and class_names are defined elsewhere
class_names = ['Hate speech', 'Offensive language', 'Neutral']

# Calculate and print average DoE
average_doe = calculate_average_doe(df, wrapped_model, class_names)
print(f"Average Degree of Explainability (DoE) for {num_sam} samples:", average_doe)